In [1]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd

# Data

### Meta & Historical

##### Filter

In [2]:
COUNTRY = "US"
SECTOR = None

##### Meta

In [3]:
meta = pd.read_csv(
    Path.cwd() / "data" / "meta.csv",
    parse_dates=["first_include"],
    date_format="%Y-%m-%d",
)

In [4]:
if COUNTRY is not None:
    meta = meta[meta['country'] == COUNTRY].reset_index(drop=True)
if SECTOR is not None:
    meta = meta[meta['gics_sector'] == SECTOR].reset_index(drop=True)

##### Historical Prices (Monthly)

In [5]:
historical = (
    pd.read_csv(Path.cwd() / "data" / "historical_prices_monthly_stat.csv")
    .sort_values(["_code", "_year", "_month"], ascending=True)
    .reset_index(drop=True)
)

##### Merge Meta & Historical Prices

In [6]:
df = pd.merge(historical, meta, how="inner", on="_code")
df["ym"] = pd.to_datetime(
    df["_year"].astype(str) + df["_month"].astype(str).str.rjust(2, "0"), 
    format="%Y%m"
)
# Only use historical price data to remove survival effect
df = df[df["ym"] >= df["first_include"]].reset_index(drop=True)

### Prices

In [7]:
prices = pd.read_csv(Path.cwd() / 'data' / 'historical_prices.csv', parse_dates=['_date'], date_format='%Y-%m-%d')
prices['_year'] = prices['_date'].dt.year
prices['_month'] = prices['_date'].dt.month
prices = prices[(prices['_date'] >= '2014-05-01') & (prices['_date'] <= '2024-03-31')]

In [8]:
prices = prices[prices['_code'].isin(df['_code'].unique())]
prices = prices.sort_values(['_code', '_date']).reset_index(drop=True)
prices['rtn'] = prices.groupby('_code', as_index=False)['_value'].pct_change(1)

### ETF

In [9]:
etf = pd.read_csv(
    Path.cwd() / "data" / "historical_prices_etf.csv",
    parse_dates=["_date"],
)
etf.set_index(['_date'], inplace=True)
etf = etf[(etf.index >= '2014-05-01') & (etf.index <= '2024-03-31')]

In [10]:
etf = etf[["ACWI-US", "VLUE-US",]]

In [11]:
eidx = etf.index

In [12]:
etf = (etf.pct_change(1).fillna(0.) + 1.).cumprod()

# Strat

### Cut Off = 0

In [13]:
cut = -0.0
cut_low = -np.inf

In [14]:
cut_df = df[(np.exp(df['monthly_high_end_rtn'])-1 <= cut) & (np.exp(df['monthly_high_end_rtn'])-1 >= cut_low)][
    ['ym', '_code', 'monthly_high_end_rtn']
].sort_values(['ym', '_code']).reset_index(drop=True)
cut_df['ym'] = cut_df['ym'] + pd.DateOffset(months=1)

In [15]:
strat = pd.DataFrame(1., columns=['strat'], index=eidx)
strat['count'] = 0.
strat['equity_weight'] = 1.

In [16]:
charge = 0.00025

In [17]:
yy = 2014
mm = 4
equity_weight = 1.

for ix, _ in tqdm(enumerate(eidx)):

    if eidx[ix].month != mm:

        mm = eidx[ix].month
        yy = eidx[ix].year

        # 월 바꼈으니 새로 편입할 종목들로 최신화
        code_list = list(cut_df[(cut_df['ym'].dt.year == yy) & (cut_df['ym'].dt.month == mm)]['_code'].unique())
        rtn_cut_df = prices[prices['_code'].isin(code_list) & (prices['_year'] == yy) & (prices['_month'] == mm)].set_index(['_date', '_code'])['rtn'].unstack(1).fillna(0.)
        code_list = list(rtn_cut_df.columns)
        # 일단 전월 주식 비중만큼 매도 수수료 계산(0.0025), 매수 수수료는 없는 샘으로 침
        if ix != 0 :
            strat.iloc[ix, 0] = strat.iloc[ix-1, 0] * equity_weight * (1. - charge) + strat.iloc[ix-1, 0] * (1. - equity_weight)
        if len(code_list) == 0:
            continue
        # 비중은 둥일 비중 또는 전월 고점 대비 월말 수익률 비로 하면 되는데... 그래도 맥스 편입비중 0.05 등으로 막아두자
        weight = np.array([np.min([1/len(code_list), 0.1]) for _ in range(len(code_list))], dtype=float) # np.min(1/len(code_list), 0.05)
        # 월 시작 주식 비중 계산(만약 각 종목의 맥스 편입비중이 존재한다면 1을 보장할 수 없음, 나머진 현금)
        equity_weight = np.sum(weight)

        # 각 종목별 현재 보유분 가치
        stock_hold = weight * strat.iloc[ix, 0]
        cash_hold = (1-equity_weight) * strat.iloc[ix, 0] # 이제부터 한달간 이놈은 항상 고정

        strat.iloc[ix, 1] = len(code_list)
        strat.iloc[ix, 2] = equity_weight
    else:
        strat.iloc[ix, 0] = strat.iloc[ix-1, 0]
        if len(code_list) == 0:
            continue
        idxstr = eidx[ix].strftime('%Y-%m-%d')
        if idxstr in rtn_cut_df.index:
            stock_hold = stock_hold * (rtn_cut_df.loc[idxstr, :].to_numpy() + 1.)
            strat.iloc[ix, 0] = np.sum(stock_hold) + cash_hold
            equity_weight = np.sum(stock_hold) / (np.sum(stock_hold) + cash_hold)
        strat.iloc[ix, 1] = len(code_list)
        strat.iloc[ix, 2] = equity_weight
            

2495it [00:13, 188.13it/s]


In [18]:
etf[f'strat_{cut}'] = strat['strat']

In [19]:
import plotly.express as px
fig = px.line(etf, x=etf.index, y=etf.columns,
              title=f'Equal Weight Portfolio ({COUNTRY})', 
              width=1000,
              height=400,
              template='plotly_white')
fig.show()

In [20]:
# import plotly.express as px
# fig = px.line(strat, x=strat.index, y='equity_weight',
#               title='custom tick labels')
# fig.show()